# Día 3: Funciones Limpias\n\n## Descripción General\n\nLas funciones son los bloques fundamentales de construcción en la programación. Escribir funciones limpias y bien diseñadas es esencial para crear código mantenible, legible y fácil de probar. En este notebook, aprenderás los principios clave para escribir funciones que sean pequeñas, enfocadas y fáciles de entender.\n\nUna función limpia hace una cosa y la hace bien. Tiene un nombre descriptivo, pocos parámetros, y es fácil de leer de arriba a abajo como una narrativa.

## Objetivos de Aprendizaje\n\nAl finalizar este notebook, serás capaz de:\n\n1. Aplicar el principio de responsabilidad única (SRP) a funciones\n2. Escribir funciones pequeñas y enfocadas que hagan una sola cosa\n3. Limitar el número de parámetros en funciones para mejorar la legibilidad\n4. Identificar y refactorizar funciones que hacen demasiado\n5. Organizar código en niveles de abstracción apropiados

## 1. El Principio de Responsabilidad Única (SRP)\n\n### 🎯 Contexto: Por Qué Importa\n\n**Problema real en Data/IA**: \n\nEstás construyendo un pipeline de ML. Tienes una función `train_model()` que: carga datos, limpia datos, entrena modelo, evalúa modelo, guarda modelo, envía métricas a dashboard, y notifica por email. Cuando falla, ¿dónde está el bug? ¿En la carga? ¿En el entrenamiento? ¿En el guardado? **Imposible saber sin debuggear todo** 💥\n\nCuando necesitas cambiar cómo guardas el modelo, tienes que tocar una función de 200 líneas que hace 7 cosas diferentes. **Alto riesgo de romper algo** 💥\n\n**Ejemplo concreto para juniors**:\n\nTienes función `process_user_data()` que valida email, normaliza nombre, guarda en DB, y envía email de bienvenida. Cuando el email falla, toda la función falla. Cuando quieres cambiar la validación, tienes que entender toda la lógica de guardado y envío de emails.\n\n**Consecuencias de NO usarlo**:\n- **Debugging difícil** → no sabes qué parte falló\n- **Testing imposible** → no puedes testear validación sin DB y email\n- **Cambios riesgosos** → modificar una parte puede romper otra\n- **No reutilizable** → no puedes usar solo la validación en otro lugar\n- **Code reviews lentos** → revisor debe entender 7 cosas a la vez\n\n### 📚 El Concepto\n\n**Definición técnica**:\n\nUna función debe tener **una sola razón para cambiar**. Debe hacer **una cosa** y hacerla bien. Si puedes describir lo que hace con \"Y\" u \"O\", hace demasiado.\n\n**Cómo identificar violaciones**:\n1. Nombre de función usa \"and\" o \"or\": `validate_and_save()` ❌\n2. Función tiene múltiples secciones con comentarios: `# Validate`, `# Transform`, `# Save` ❌\n3. Función tiene múltiples niveles de indentación (>3) ❌\n4. No puedes testear una parte sin las otras ❌\n5. Cambiar una cosa requiere entender toda la función ❌\n\n**Terminología clave**:\n- **Single Responsibility**: Una sola razón para cambiar\n- **Cohesión**: Qué tan relacionadas están las operaciones\n- **Acoplamiento**: Qué tan dependientes son las partes\n- **Separation of Concerns**: Separar diferentes preocupaciones

In [ ]:
# BAD: Function doing too many things\ndef process_user_data(user_dict):\n    \"\"\"Process user data - validates, transforms, saves, and sends email.\"\"\"\n    # Validate\n    if not user_dict.get('email'):\n        raise ValueError('Email required')\n    if '@' not in user_dict['email']:\n        raise ValueError('Invalid email')\n    \n    # Transform\n    user_dict['email'] = user_dict['email'].lower().strip()\n    user_dict['name'] = user_dict.get('name', '').title()\n    \n    # Save to database\n    db_connection = get_database_connection()\n    db_connection.execute('INSERT INTO users VALUES (?)', user_dict)\n    \n    # Send welcome email\n    email_client = get_email_client()\n    email_client.send(user_dict['email'], 'Welcome!')\n    \n    return user_dict

In [ ]:
# GOOD: Each function has a single responsibility\ndef validate_user_email(email: str) -> str:\n    \"\"\"\n    Validate and normalize email address.\n    \n    :param email: Email address to validate\n    :type email: str\n    :return: Normalized email address\n    :rtype: str\n    :raises ValueError: If email is invalid\n    \"\"\"\n    if not email:\n        raise ValueError('Email required')\n    if '@' not in email:\n        raise ValueError('Invalid email')\n    return email.lower().strip()\n\n\ndef normalize_user_name(name: str) -> str:\n    \"\"\"\n    Normalize user name to title case.\n    \n    :param name: User name\n    :type name: str\n    :return: Normalized name\n    :rtype: str\n    \"\"\"\n    return name.title() if name else ''\n\n\ndef save_user_to_database(user_data: dict) -> None:\n    \"\"\"\n    Save user data to database.\n    \n    :param user_data: User information\n    :type user_data: dict\n    \"\"\"\n    db_connection = get_database_connection()\n    db_connection.execute('INSERT INTO users VALUES (?)', user_data)\n\n\ndef send_welcome_email(email: str) -> None:\n    \"\"\"\n    Send welcome email to new user.\n    \n    :param email: User email address\n    :type email: str\n    \"\"\"\n    email_client = get_email_client()\n    email_client.send(email, 'Welcome!')\n\n\ndef process_user_data(user_dict: dict) -> dict:\n    \"\"\"\n    Process new user registration.\n    \n    :param user_dict: Raw user data\n    :type user_dict: dict\n    :return: Processed user data\n    :rtype: dict\n    \"\"\"\n    user_dict['email'] = validate_user_email(user_dict.get('email', ''))\n    user_dict['name'] = normalize_user_name(user_dict.get('name', ''))\n    save_user_to_database(user_dict)\n    send_welcome_email(user_dict['email'])\n    return user_dict

### 💡 Aprendizaje Clave\n\n**Puntos críticos a recordar**:\n1. Una función = **una razón para cambiar** (no 7 razones)\n2. Si usas \"Y\" u \"O\" para describir → **divide la función**\n3. Comentarios que separan secciones → **cada sección es una función**\n4. No puedes testear una parte sin las otras → **mal diseño**\n\n**Cómo desarrollar intuición**:\n- **Pregúntate**: \"¿Cuántas razones tiene esta función para cambiar?\"\n  - 1 razón → perfecto ✅\n  - 2+ razones → divide en funciones ❌\n\n- **Pregúntate**: \"¿Puedo testear cada parte independientemente?\"\n  - SÍ → buen diseño ✅\n  - NO → funciones acopladas ❌\n\n**Cuándo usar / NO usar**:\n- ✅ **SIEMPRE aplica SRP**:\n  - Funciones de negocio\n  - Funciones de procesamiento\n  - Funciones de validación\n  - Funciones de transformación\n  \n- ⚠️ **Excepción rara**:\n  - Scripts one-off muy simples\n  - Prototipos rápidos (pero refactoriza después)\n\n**Patrón de refactorización**:\n```python\n# Antes: 1 función hace 4 cosas\ndef process_data(data):\n    # Validate (razón 1)\n    # Transform (razón 2)\n    # Save (razón 3)\n    # Notify (razón 4)\n    pass\n\n# Después: 4 funciones, cada una hace 1 cosa\ndef validate_data(data): ...\ndef transform_data(data): ...\ndef save_data(data): ...\ndef notify_completion(data): ...\n\ndef process_data(data):\n    validated = validate_data(data)\n    transformed = transform_data(validated)\n    save_data(transformed)\n    notify_completion(transformed)\n```\n\n**Beneficios medibles**:\n- **Testing**: 4 tests simples vs 1 test complejo\n- **Debugging**: Sabes exactamente qué función falló\n- **Reutilización**: Puedes usar `validate_data()` en otro lugar\n- **Cambios**: Modificar validación no afecta guardado\n\n**Referencia oficial:** [PEP 8 - Style Guide for Python Code](https://peps.python.org/pep-0008/)

## 2. Funciones Pequeñas\n\n### 🎯 Contexto: Por Qué Importa\n\n**Problema real en Data/IA**: \n\nEstás revisando código de un colega. Encuentras función `preprocess_dataset()` de **150 líneas**. Tiene 8 niveles de indentación, 15 variables locales, y hace: carga datos, maneja missing values, normaliza features, hace encoding, split train/test, aplica transformaciones, valida resultados, y guarda outputs. **Tardas 30 minutos en entender qué hace** 💥\n\nHay un bug en la normalización. Para arreglarlo, tienes que entender toda la función porque las variables se reutilizan. **Alto riesgo de romper algo** 💥\n\n**Ejemplo concreto para juniors**:\n\nFunción `generate_report()` de 80 líneas: parsea datos, calcula estadísticas, agrupa por categoría, formatea output, genera gráficos, y guarda archivo. Cuando necesitas cambiar el formato, tienes que leer 80 líneas para encontrar las 5 líneas relevantes.\n\n**Consecuencias de NO usarlo**:\n- **Comprensión lenta** → 30 min para entender vs 2 min\n- **Bugs ocultos** → difícil ver la lógica completa\n- **Testing imposible** → no puedes testear partes individuales\n- **Reutilización cero** → no puedes extraer partes útiles\n- **Onboarding lento** → nuevos devs tardan días en entender\n\n### 📚 El Concepto\n\n**Definición técnica**:\n\nFunciones deben ser **pequeñas**. Idealmente **< 20 líneas**, máximo **< 50 líneas**. Si es más larga, probablemente hace múltiples cosas y debe dividirse.\n\n**Reglas de tamaño**:\n1. **Ideal**: 5-20 líneas\n2. **Aceptable**: 20-50 líneas\n3. **Refactorizar**: > 50 líneas\n4. **Alerta roja**: > 100 líneas\n\n**Señales de función demasiado larga**:\n- Más de 3 niveles de indentación\n- Múltiples secciones con comentarios\n- Scroll necesario para ver toda la función\n- Múltiples variables temporales\n- Difícil nombrar sin usar \"and\"\n\n**Terminología clave**:\n- **Función pequeña**: < 20 líneas, hace una cosa\n- **Función grande**: > 50 líneas, probablemente hace múltiples cosas\n- **Complejidad ciclomática**: Número de caminos de ejecución\n- **Niveles de indentación**: Profundidad de anidamiento

In [ ]:
# BAD: Long function with multiple levels of abstraction\ndef generate_report(data):\n    \"\"\"Generate a comprehensive report from data.\"\"\"\n    # Parse data\n    parsed = []\n    for item in data:\n        if item.get('valid'):\n            parsed.append({\n                'id': item['id'],\n                'value': float(item['value']),\n                'category': item.get('category', 'unknown')\n            })\n    \n    # Calculate statistics\n    total = sum(item['value'] for item in parsed)\n    average = total / len(parsed) if parsed else 0\n    by_category = {}\n    for item in parsed:\n        cat = item['category']\n        if cat not in by_category:\n            by_category[cat] = []\n        by_category[cat].append(item['value'])\n    \n    # Format output\n    report = f\"Total: {total}\\n\"\n    report += f\"Average: {average:.2f}\\n\"\n    report += \"\\nBy Category:\\n\"\n    for cat, values in by_category.items():\n        cat_avg = sum(values) / len(values)\n        report += f\"  {cat}: {cat_avg:.2f}\\n\"\n    \n    return report

In [ ]:
# GOOD: Small, focused functions\ndef parse_valid_items(data: list[dict]) -> list[dict]:\n    \"\"\"\n    Extract and normalize valid items from raw data.\n    \n    :param data: Raw data items\n    :type data: list[dict]\n    :return: Parsed valid items\n    :rtype: list[dict]\n    \"\"\"\n    return [\n        {\n            'id': item['id'],\n            'value': float(item['value']),\n            'category': item.get('category', 'unknown')\n        }\n        for item in data\n        if item.get('valid')\n    ]\n\n\ndef calculate_statistics(items: list[dict]) -> dict:\n    \"\"\"\n    Calculate total and average from items.\n    \n    :param items: Parsed items\n    :type items: list[dict]\n    :return: Statistics dictionary\n    :rtype: dict\n    \"\"\"\n    values = [item['value'] for item in items]\n    total = sum(values)\n    average = total / len(values) if values else 0\n    return {'total': total, 'average': average}\n\n\ndef group_by_category(items: list[dict]) -> dict[str, list[float]]:\n    \"\"\"\n    Group item values by category.\n    \n    :param items: Parsed items\n    :type items: list[dict]\n    :return: Values grouped by category\n    :rtype: dict[str, list[float]]\n    \"\"\"\n    by_category = {}\n    for item in items:\n        category = item['category']\n        by_category.setdefault(category, []).append(item['value'])\n    return by_category\n\n\ndef format_report(stats: dict, by_category: dict[str, list[float]]) -> str:\n    \"\"\"\n    Format statistics into a readable report.\n    \n    :param stats: Overall statistics\n    :type stats: dict\n    :param by_category: Values by category\n    :type by_category: dict[str, list[float]]\n    :return: Formatted report\n    :rtype: str\n    \"\"\"\n    lines = [\n        f\"Total: {stats['total']}\",\n        f\"Average: {stats['average']:.2f}\",\n        \"\\nBy Category:\"\n    ]\n    \n    for category, values in by_category.items():\n        avg = sum(values) / len(values)\n        lines.append(f\"  {category}: {avg:.2f}\")\n    \n    return '\\n'.join(lines)\n\n\ndef generate_report(data: list[dict]) -> str:\n    \"\"\"\n    Generate a comprehensive report from data.\n    \n    :param data: Raw data items\n    :type data: list[dict]\n    :return: Formatted report\n    :rtype: str\n    \"\"\"\n    items = parse_valid_items(data)\n    stats = calculate_statistics(items)\n    by_category = group_by_category(items)\n    return format_report(stats, by_category)

### 💡 Aprendizaje Clave\n\n**Puntos críticos a recordar**:\n1. **< 20 líneas** es ideal, **< 50 líneas** es aceptable\n2. Si necesitas **scroll** para ver toda la función → demasiado larga\n3. **> 3 niveles de indentación** → extrae funciones\n4. Comentarios que separan secciones → **cada sección es una función**\n\n**Cómo desarrollar intuición**:\n- **Pregúntate**: \"¿Puedo ver toda la función sin scroll?\"\n  - SÍ → tamaño OK ✅\n  - NO → divide en funciones ❌\n\n- **Pregúntate**: \"¿Entiendo qué hace en 30 segundos?\"\n  - SÍ → bien diseñada ✅\n  - NO → demasiado compleja ❌\n\n**Cuándo usar / NO usar**:\n- ✅ **SIEMPRE mantén funciones pequeñas**:\n  - Funciones de negocio\n  - Funciones de procesamiento\n  - Funciones de transformación\n  - Funciones de validación\n  \n- ⚠️ **Excepción rara**:\n  - Funciones con muchos casos edge (pero considera tabla de decisiones)\n  - Scripts one-off (pero refactoriza si reutilizas)\n\n**Técnicas de división**:\n```python\n# Técnica 1: Extraer por secciones\ndef big_function():\n    # Section 1: Parse → extract_parse()\n    # Section 2: Calculate → extract_calculate()\n    # Section 3: Format → extract_format()\n    pass\n\n# Técnica 2: Extraer por niveles de indentación\ndef big_function():\n    for item in items:\n        if condition:\n            # Nested logic → extract_process_item()\n            pass\n\n# Técnica 3: Extraer cálculos complejos\ndef big_function():\n    result = (a * b + c) / (d - e) * f  # → extract_calculate_metric()\n```\n\n**Beneficios medibles**:\n- **Comprensión**: 2 min vs 30 min\n- **Testing**: 5 tests simples vs 1 test complejo\n- **Debugging**: Sabes exactamente dónde está el bug\n- **Reutilización**: Puedes usar funciones pequeñas en otros lugares\n\n**Herramientas para medir**:\n```python\n# Usar pylint para detectar funciones largas\n# pylint: disable=too-many-lines\n\n# Usar radon para medir complejidad\n# radon cc myfile.py -s\n```\n\n**Referencia oficial:** [Clean Code by Robert C. Martin](https://www.oreilly.com/library/view/clean-code-a/9780136083238/)

## 3. Limitar el Número de Parámetros\n\n### 🎯 Contexto: Por Qué Importa\n\n**Problema real en Data/IA**: \n\nEncuentras función:\n```python\ntrain_model(data, target, test_size, random_state, n_estimators, \n            max_depth, min_samples_split, min_samples_leaf, \n            max_features, bootstrap, oob_score, n_jobs, verbose)\n```\n\n**13 parámetros** 💥 ¿En qué orden van? ¿Cuáles son obligatorios? ¿Qué valores por defecto tienen? **Imposible recordar** 💥\n\nCada llamada es una pesadilla:\n```python\ntrain_model(X, y, 0.2, 42, 100, 10, 2, 1, 'sqrt', True, False, -1, 0)\n```\n¿Qué significa cada número? **Nadie sabe sin ver la definición** 💥\n\n**Ejemplo concreto para juniors**:\n\nFunción `create_user(first, last, email, phone, address, city, state, zip, country, birth, gender)` con 11 parámetros. Cada vez que la llamas, tienes que buscar el orden correcto. Cuando añades un parámetro, rompes todas las llamadas existentes.\n\n**Consecuencias de NO usarlo**:\n- **Llamadas ilegibles** → nadie entiende qué hace cada argumento\n- **Errores frecuentes** → parámetros en orden incorrecto\n- **Testing pesadilla** → necesitas mockear 13 parámetros\n- **Cambios imposibles** → añadir parámetro rompe todo\n- **Documentación compleja** → 13 parámetros para documentar\n\n### 📚 El Concepto\n\n**Definición técnica**:\n\nLimita funciones a **3 parámetros o menos**. Si necesitas más, agrupa parámetros relacionados en objetos (dataclasses, dicts, namedtuples).\n\n**Reglas de parámetros**:\n1. **Ideal**: 0-1 parámetros (niladic, monadic)\n2. **Bueno**: 2 parámetros (dyadic)\n3. **Aceptable**: 3 parámetros (triadic)\n4. **Refactorizar**: > 3 parámetros\n5. **Alerta roja**: > 5 parámetros\n\n**Técnicas para reducir parámetros**:\n1. **Dataclasses**: Agrupa parámetros relacionados\n2. **Config objects**: Usa objeto de configuración\n3. **Builder pattern**: Construye objeto paso a paso\n4. **Kwargs**: Para opciones opcionales\n5. **Method chaining**: Encadena llamadas\n\n**Terminología clave**:\n- **Niladic**: 0 parámetros\n- **Monadic**: 1 parámetro\n- **Dyadic**: 2 parámetros\n- **Triadic**: 3 parámetros\n- **Polyadic**: > 3 parámetros (evitar)

In [ ]:
# BAD: Too many parameters\ndef create_user(first_name, last_name, email, phone, address, city, \n                state, zip_code, country, birth_date, gender):\n    \"\"\"Create a new user with all details.\"\"\"\n    # Implementation\n    pass

In [ ]:
# GOOD: Use a data class to group related parameters\nfrom dataclasses import dataclass\nfrom datetime import date\n\n\n@dataclass\nclass UserProfile:\n    \"\"\"User profile information.\"\"\"\n    first_name: str\n    last_name: str\n    email: str\n    phone: str\n    \n\n@dataclass\nclass Address:\n    \"\"\"User address information.\"\"\"\n    street: str\n    city: str\n    state: str\n    zip_code: str\n    country: str\n    \n\n@dataclass\nclass PersonalInfo:\n    \"\"\"Personal information.\"\"\"\n    birth_date: date\n    gender: str\n\n\ndef create_user(profile: UserProfile, address: Address, \n                personal_info: PersonalInfo) -> dict:\n    \"\"\"\n    Create a new user with organized information.\n    \n    :param profile: User profile data\n    :type profile: UserProfile\n    :param address: Address data\n    :type address: Address\n    :param personal_info: Personal information\n    :type personal_info: PersonalInfo\n    :return: Created user data\n    :rtype: dict\n    \"\"\"\n    # Implementation\n    return {\n        'profile': profile,\n        'address': address,\n        'personal_info': personal_info\n    }

### 💡 Aprendizaje Clave\n\n**Puntos críticos a recordar**:\n1. **≤ 3 parámetros** es la regla de oro\n2. **> 3 parámetros** → agrupa en objetos\n3. **Parámetros booleanos** → divide en 2 funciones\n4. **Orden importa** → parámetros más importantes primero\n\n**Cómo desarrollar intuición**:\n- **Pregúntate**: \"¿Puedo recordar el orden de los parámetros?\"\n  - SÍ (≤ 3) → OK ✅\n  - NO (> 3) → agrupa en objeto ❌\n\n- **Pregúntate**: \"¿Los parámetros están relacionados?\"\n  - SÍ → agrupa en dataclass ✅\n  - NO → tal vez la función hace demasiado ❌\n\n**Cuándo usar / NO usar**:\n- ✅ **Usar agrupación cuando**:\n  - > 3 parámetros relacionados\n  - Parámetros siempre van juntos\n  - Necesitas añadir más parámetros en el futuro\n  - Parámetros forman concepto cohesivo\n  \n- ❌ **NO agrupar cuando**:\n  - Solo 2-3 parámetros simples\n  - Parámetros no relacionados\n  - Agrupación añade complejidad innecesaria\n\n**Patrones de refactorización**:\n```python\n# Patrón 1: Dataclass para parámetros relacionados\n@dataclass\nclass UserInfo:\n    first_name: str\n    last_name: str\n    email: str\n\ndef create_user(info: UserInfo, address: Address): ...\n\n# Patrón 2: Config object\n@dataclass\nclass ModelConfig:\n    n_estimators: int = 100\n    max_depth: int = 10\n    random_state: int = 42\n\ndef train_model(data, config: ModelConfig): ...\n\n# Patrón 3: Builder pattern\nmodel = ModelBuilder()\\\n    .with_estimators(100)\\\n    .with_depth(10)\\\n    .build()\n\n# Patrón 4: Kwargs para opciones\ndef process_data(data, **options): ...\n```\n\n**Antipatrón: Parámetro booleano**:\n```python\n# ❌ MAL: Flag parameter\ndef save_data(data, compress=False):\n    if compress:\n        # Logic A\n    else:\n        # Logic B\n\n# ✅ BIEN: Dos funciones\ndef save_data(data): ...\ndef save_compressed_data(data): ...\n```\n\n**Beneficios medibles**:\n- **Legibilidad**: Llamadas auto-documentadas\n- **Mantenibilidad**: Añadir parámetro no rompe código\n- **Testing**: Menos combinaciones para testear\n- **Refactoring**: Más fácil cambiar estructura\n\n**Referencia oficial:** [Python dataclasses](https://docs.python.org/3/library/dataclasses.html)

## 4. Niveles de Abstracción\n\n### 🎯 Contexto: Por Qué Importa\n\n**Problema real en Data/IA**: \n\nLees función `process_orders()`:\n```python\ndef process_orders(orders):\n    validate_orders(orders)  # Alto nivel\n    \n    # Bajo nivel mezclado\n    total = 0\n    for item in order['items']:\n        price = item['price']\n        qty = item['quantity']\n        disc = item.get('discount', 0)\n        total += price * qty * (1 - disc)\n    \n    charge_customer(total)  # Alto nivel otra vez\n```\n\nTu cerebro salta entre **conceptos de alto nivel** (validar, cobrar) y **detalles de bajo nivel** (loops, cálculos). **Difícil seguir la narrativa** 💥\n\n**Ejemplo concreto para juniors**:\n\nFunción que mezcla: llamadas a APIs (alto nivel), manipulación de strings (bajo nivel), lógica de negocio (medio nivel), y operaciones de bits (muy bajo nivel). Tu cerebro no puede mantener todos los niveles a la vez.\n\n**Consecuencias de NO usarlo**:\n- **Comprensión lenta** → cerebro salta entre niveles\n- **Narrativa rota** → no fluye como historia\n- **Mantenimiento difícil** → cambios en bajo nivel afectan alto nivel\n- **Testing complejo** → no puedes testear niveles independientemente\n\n### 📚 El Concepto\n\n**Definición técnica**:\n\nCada función debe operar en **un solo nivel de abstracción**. No mezcles operaciones de alto nivel con detalles de bajo nivel.\n\n**Niveles de abstracción**:\n1. **Muy alto**: Conceptos de negocio (`process_payment`, `send_notification`)\n2. **Alto**: Operaciones de dominio (`validate_email`, `calculate_total`)\n3. **Medio**: Transformaciones (`parse_json`, `format_date`)\n4. **Bajo**: Operaciones básicas (`split`, `lower`, `append`)\n5. **Muy bajo**: Operaciones de bits, memoria\n\n**Regla de lectura (Stepdown Rule)**:\nCódigo debe leerse como narrativa de arriba a abajo, cada función un nivel más bajo que la anterior:\n```\nprocess_orders()        # Nivel 1\n  ├─ validate_orders()  # Nivel 2\n  ├─ calculate_total()  # Nivel 2\n  └─ charge_customer()  # Nivel 2\n```\n\n**Terminología clave**:\n- **Abstracción**: Ocultar detalles de implementación\n- **Nivel de abstracción**: Qué tan cerca estás del hardware/detalles\n- **Stepdown rule**: Leer código como bajando escaleras\n- **Leaky abstraction**: Cuando detalles de bajo nivel se filtran

In [ ]:
# BAD: Mixed abstraction levels\ndef process_orders(orders):\n    \"\"\"Process customer orders.\"\"\"\n    for order in orders:\n        # High-level concept\n        validate_order(order)\n        \n        # Low-level details mixed in\n        total = 0\n        for item in order['items']:\n            price = item['price']\n            quantity = item['quantity']\n            discount = item.get('discount', 0)\n            total += price * quantity * (1 - discount)\n        \n        # High-level concept again\n        charge_customer(order['customer_id'], total)\n        send_confirmation(order)

In [ ]:
# GOOD: Consistent abstraction level\ndef calculate_order_total(order: dict) -> float:\n    \"\"\"\n    Calculate total price for an order.\n    \n    :param order: Order data\n    :type order: dict\n    :return: Total price\n    :rtype: float\n    \"\"\"\n    return sum(\n        item['price'] * item['quantity'] * (1 - item.get('discount', 0))\n        for item in order['items']\n    )\n\n\ndef process_single_order(order: dict) -> None:\n    \"\"\"\n    Process a single customer order.\n    \n    :param order: Order data\n    :type order: dict\n    \"\"\"\n    validate_order(order)\n    total = calculate_order_total(order)\n    charge_customer(order['customer_id'], total)\n    send_confirmation(order)\n\n\ndef process_orders(orders: list[dict]) -> None:\n    \"\"\"\n    Process multiple customer orders.\n    \n    :param orders: List of orders\n    :type orders: list[dict]\n    \"\"\"\n    for order in orders:\n        process_single_order(order)

### 💡 Aprendizaje Clave\n\n**Puntos críticos a recordar**:\n1. **Un nivel por función** → no mezcles alto y bajo nivel\n2. **Stepdown rule** → lee como bajando escaleras\n3. **Extrae detalles** → detalles de bajo nivel en funciones separadas\n4. **Narrativa clara** → debe fluir como historia\n\n**Cómo desarrollar intuición**:\n- **Pregúntate**: \"¿Todas las operaciones están al mismo nivel?\"\n  - SÍ → bien diseñada ✅\n  - NO → extrae niveles diferentes ❌\n\n- **Pregúntate**: \"¿Puedo leer esto como una historia?\"\n  - SÍ → niveles consistentes ✅\n  - NO → niveles mezclados ❌\n\n**Cuándo usar / NO usar**:\n- ✅ **SIEMPRE mantén niveles consistentes**:\n  - Funciones de negocio\n  - Funciones de procesamiento\n  - Funciones de coordinación\n  - Funciones públicas de API\n  \n- ⚠️ **Excepción rara**:\n  - Funciones de utilidad muy simples\n  - Performance crítico (pero documenta por qué)\n\n**Patrón de refactorización**:\n```python\n# ❌ MAL: Niveles mezclados\ndef process_data(data):\n    validate(data)  # Alto nivel\n    \n    # Bajo nivel mezclado\n    result = []\n    for item in data:\n        if item['value'] > 0:\n            result.append(item['value'] * 2)\n    \n    save(result)  # Alto nivel\n\n# ✅ BIEN: Niveles consistentes\ndef process_data(data):\n    validate(data)\n    transformed = transform(data)  # Mismo nivel\n    save(transformed)\n\ndef transform(data):\n    return [\n        item['value'] * 2\n        for item in data\n        if item['value'] > 0\n    ]\n```\n\n**Test de niveles**:\n```python\n# Pregunta: ¿Qué nivel es cada operación?\n\n# Alto nivel\nprocess_payment()\nsend_notification()\n\n# Medio nivel\nvalidate_email()\ncalculate_total()\n\n# Bajo nivel\nemail.lower().strip()\nsum(prices)\n\n# Si mezclas en una función → problema\n```\n\n**Beneficios medibles**:\n- **Comprensión**: 5x más rápido entender\n- **Mantenimiento**: Cambios localizados\n- **Testing**: Testea cada nivel independientemente\n- **Reutilización**: Funciones de bajo nivel reutilizables\n\n**Analogía útil**:\nComo leer un libro:\n- Capítulos (alto nivel)\n- Secciones (medio nivel)\n- Párrafos (bajo nivel)\n- Oraciones (muy bajo nivel)\n\nNo mezclas oraciones con títulos de capítulos.\n\n**Referencia oficial:** [The Stepdown Rule - Clean Code](https://www.oreilly.com/library/view/clean-code-a/9780136083238/)

## Ejercicios Prácticos\n\nAhora es tu turno de practicar escribiendo funciones limpias. Completa los ejercicios en el archivo `exercises/01_clean_functions.py` y verifica tus soluciones con `pytest tests/test_01_clean_functions.py`.

### Ejercicio 1: Refactorizar Función con Múltiples Responsabilidades\n\nRefactoriza la siguiente función que hace demasiadas cosas en funciones más pequeñas y enfocadas.

In [ ]:
# Ver exercises/01_clean_functions.py para el ejercicio completo

### Ejercicio 2: Reducir Parámetros\n\nRefactoriza una función con muchos parámetros usando dataclasses o diccionarios.

In [ ]:
# Ver exercises/01_clean_functions.py para el ejercicio completo

### Ejercicio 3: Separar Niveles de Abstracción\n\nReorganiza una función que mezcla diferentes niveles de abstracción.

In [ ]:
# Ver exercises/01_clean_functions.py para el ejercicio completo

## Resumen\n\nEn este notebook has aprendido los principios fundamentales para escribir funciones limpias:\n\n1. **Responsabilidad Única**: Cada función debe hacer una cosa y hacerla bien\n2. **Funciones Pequeñas**: Mantén las funciones cortas (idealmente < 20 líneas)\n3. **Pocos Parámetros**: Limita a 3 parámetros o menos, agrupa parámetros relacionados\n4. **Niveles de Abstracción**: Mantén un nivel de abstracción consistente en cada función\n5. **Nombres Descriptivos**: El nombre debe describir claramente lo que hace la función\n\nEstos principios te ayudarán a escribir código más mantenible, testeable y fácil de entender.

## Preguntas de Autoevaluación\n\n### 1. ¿Qué significa que una función tenga una sola responsabilidad?\n\n**Respuesta:** Significa que la función debe tener una sola razón para cambiar. Debe hacer una cosa específica y hacerla bien, sin mezclar múltiples tareas o responsabilidades.\n\n### 2. ¿Cuál es el número ideal de parámetros para una función?\n\n**Respuesta:** Idealmente 3 o menos. Si necesitas más parámetros, considera agrupar parámetros relacionados en objetos (dataclasses, namedtuples) o usar **kwargs para opciones.\n\n### 3. ¿Cómo identificas que una función es demasiado larga?\n\n**Respuesta:** Si la función tiene más de 20 líneas, si hace múltiples cosas que podrías describir con \"y\" u \"o\", o si no puedes entender qué hace sin leer todo el código, probablemente es demasiado larga.\n\n### 4. ¿Qué problema resuelve mantener niveles de abstracción consistentes?\n\n**Respuesta:** Evita que el lector tenga que saltar entre detalles de bajo nivel y conceptos de alto nivel, haciendo el código más fácil de leer y entender como una narrativa coherente.\n\n### 5. ¿Cuándo deberías dividir una función en funciones más pequeñas?\n\n**Respuesta:** Cuando la función hace múltiples cosas, cuando es difícil de testear, cuando tiene múltiples niveles de indentación, o cuando no puedes darle un nombre descriptivo sin usar \"y\" u \"o\".\n\n### 6. ¿Qué ventajas tiene escribir funciones pequeñas y enfocadas?\n\n**Respuesta:** Son más fáciles de entender, testear, reutilizar y mantener. También facilitan la identificación de bugs y la refactorización del código.

## Recursos y Referencias Oficiales\n\n### Documentación Oficial\n- **[PEP 8 - Style Guide for Python Code](https://peps.python.org/pep-0008/)**: Guía oficial de estilo para Python\n  - Convenciones de nomenclatura y estructura de código\n\n- **[Python dataclasses](https://docs.python.org/3/library/dataclasses.html)**: Documentación oficial de dataclasses\n  - Útil para agrupar parámetros relacionados\n\n### Libros y Recursos\n- **[Clean Code by Robert C. Martin](https://www.oreilly.com/library/view/clean-code-a/9780136083238/)**: Referencia fundamental sobre código limpio\n  - Capítulo 3: Functions\n\n- **[Refactoring by Martin Fowler](https://refactoring.com/)**: Técnicas de refactorización\n  - Extract Function, Introduce Parameter Object\n\n- **[Python Clean Code](https://testdriven.io/blog/clean-code-python/)**: Guía de código limpio específica para Python\n  - Ejemplos prácticos en Python\n\n### Herramientas\n- **[Ruff](https://docs.astral.sh/ruff/)**: Linter y formateador rápido para Python\n  - Detecta funciones complejas y problemas de estilo\n\n- **[pylint](https://pylint.pycqa.org/)**: Analizador de código Python\n  - Detecta funciones con demasiados parámetros o líneas\n\n### Notas Importantes\n- Todos los enlaces están actualizados a partir de 2025\n- Se recomienda revisar la documentación oficial regularmente\n- Practica estos principios en tu código diario para internalizarlos